In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

#New iports for windowing
from pyspark.sql.functions import window


sc = SparkContext()

sc.setLogLevel("ERROR")

In [ ]:
spark = SparkSession\
    .builder\
        .appName("ContarPalavarasEstruturado")\
            .getOrCreate()

In [ ]:
linhas = spark.readStream\
    .format("socket")\
        .option("host", "localhost")\
            .option("port", 3456)\
                .option("includeTimestamp", "true")\
                .load()


In [ ]:
palavras = linhas.select(
    explode(split(linhas.value, " ")).alias("palavra"),
    linhas.timestamp
)

In [ ]:
contagem = palavras.groupBy(
    window(palavras.timestamp, "10 minutes", "5 minutes"),
    palavras.palavra
).count().orderBy('window')


In [ ]:
consulta = contagem.writeStream\
    .outputMode("complete")\
        .format("console")\
            .start()

In [ ]:
consulta.awaitTermination().option("truncate", "false")